In [1]:
!nvidia-smi

Mon May 27 07:34:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

2024-05-27 07:34:08.380791: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 07:34:08.380900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 07:34:08.636313: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# re-size all the images to this
IMAGE_SIZE = [640, 640]

train_path = '/kaggle/input/drone-data1212/train'
valid_path = '/kaggle/input/drone-data1212/valid'

In [4]:
#image size is 244,244,3
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [6]:
  # useful for getting number of output classes
folders = glob('/kaggle/input/drone-data1212/train/*')

In [7]:
folders

['/kaggle/input/drone-data1212/train/non drone',
 '/kaggle/input/drone-data1212/train/drone']

In [8]:
# our layers - you can add more if you want
x = Flatten()(inception.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [9]:
# view the structure of the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 640, 640,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 319, 319,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 319, 319,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 319, 319,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 317, 317,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 317, 317,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 317, 317,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 317, 317,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 317, 317,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 317, 317,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 158, 158,  │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 158, 158,  │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 158, 158,  │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 158, 158,  │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 156, 156,  │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 156, 156,  │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 156, 156,  │          0 │ batch_normalizat

 Total params: 23,129,890 (88.23 MB)

 Trainable params: 1,327,106 (5.06 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [10]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/kaggle/input/drone-data1212/train',
                                                 target_size = (640, 640),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 3711 images belonging to 2 classes.


In [13]:
test_set = test_datagen.flow_from_directory('/kaggle/input/drone-data1212/valid',
                                            target_size = (640, 640),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 245 images belonging to 2 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1716795363.609740     111 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1716795363.658647     111 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


 33/232 ━━━━━━━━━━━━━━━━━━━━ 4:33 1s/step - accuracy: 0.5930 - loss: 29.0203

In [18]:
model.save('inception.h5')

**Vgg16**

In [11]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [12]:
#image size is 244,244,3
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False
# our layers - you can add more if you want
x = Flatten()(resnet.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)
# view the structure of the model
#model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [14]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/25


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1714736485.929298     151 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1714736485.978597     151 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5694 - loss: 38.7468

W0000 00:00:1714736819.207696     151 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


232/232 ━━━━━━━━━━━━━━━━━━━━ 389s 2s/step - accuracy: 0.5695 - loss: 38.6723 - val_accuracy: 0.5510 - val_loss: 21.9984
Epoch 2/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 198us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/25


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


232/232 ━━━━━━━━━━━━━━━━━━━━ 311s 1s/step - accuracy: 0.6083 - loss: 18.8170 - val_accuracy: 0.5878 - val_loss: 22.7774
Epoch 4/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 84us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 313s 1s/step - accuracy: 0.6745 - loss: 12.8880 - val_accuracy: 0.6694 - val_loss: 11.2151
Epoch 6/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 82us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 309s 1s/step - accuracy: 0.6175 - loss: 40.1163 - val_accuracy: 0.7020 - val_loss: 14.1420
Epoch 8/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 80us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 307s 1s/step - accuracy: 0.7331 - loss: 13.1244 - val_accuracy: 0.6408 - val_loss: 20.7797
Epoch 10/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 77us/ste

In [16]:
model.save('resnet50.h5')

In [15]:
from tensorflow.keras.applications.mobilenet import MobileNet

In [17]:
#image size is 244,244,3
mobile_net = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# don't train existing weights
for layer in mobile_net.layers:
    layer.trainable = False
# our layers - you can add more if you want
x = Flatten()(mobile_net.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=mobile_net.input, outputs=prediction)
# view the structure of the model
#model.summary()

In [18]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [19]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/25


W0000 00:00:1714740891.337162     151 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5659 - loss: 36.2186

W0000 00:00:1714741190.499009     148 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


232/232 ━━━━━━━━━━━━━━━━━━━━ 325s 1s/step - accuracy: 0.5660 - loss: 36.1673 - val_accuracy: 0.4980 - val_loss: 43.8085
Epoch 2/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 188us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/25


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


232/232 ━━━━━━━━━━━━━━━━━━━━ 308s 1s/step - accuracy: 0.6408 - loss: 16.8995 - val_accuracy: 0.6612 - val_loss: 11.9233
Epoch 4/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 81us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 307s 1s/step - accuracy: 0.6430 - loss: 21.6392 - val_accuracy: 0.6694 - val_loss: 12.7528
Epoch 6/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 83us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 309s 1s/step - accuracy: 0.6929 - loss: 17.9113 - val_accuracy: 0.6204 - val_loss: 22.2117
Epoch 8/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 76us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 308s 1s/step - accuracy: 0.7105 - loss: 14.6716 - val_accuracy: 0.6735 - val_loss: 17.2366
Epoch 10/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 66us/ste

In [20]:
model.save('mobilenet.h5')

In [26]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

In [27]:
#image size is 244,244,3
insresnet = InceptionResNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# don't train existing weights
for layer in insresnet.layers:
    layer.trainable = False
# our layers - you can add more if you want
x = Flatten()(insresnet.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=insresnet.input, outputs=prediction)
# view the structure of the model
#model.summary()

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [28]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [29]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 484s 2s/step - accuracy: 0.8870 - loss: 2.7867 - val_accuracy: 0.9265 - val_loss: 3.8024
Epoch 2/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/25


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


232/232 ━━━━━━━━━━━━━━━━━━━━ 318s 1s/step - accuracy: 0.9708 - loss: 1.0263 - val_accuracy: 0.9265 - val_loss: 5.0089
Epoch 4/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 127us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 318s 1s/step - accuracy: 0.9853 - loss: 0.2641 - val_accuracy: 0.9265 - val_loss: 5.3270
Epoch 6/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 128us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 317s 1s/step - accuracy: 0.9804 - loss: 0.7180 - val_accuracy: 0.9143 - val_loss: 6.9258
Epoch 8/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 123us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 319s 1s/step - accuracy: 0.9866 - loss: 0.5379 - val_accuracy: 0.9306 - val_loss: 7.8147
Epoch 10/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 130us/step - 

In [30]:
model.save('inception_resnetv2.h5')

In [9]:
from tensorflow.keras.applications.nasnet import NASNetMobile

In [12]:
#image size is 244,244,3
nasnet_mobile = NASNetMobile(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# don't train existing weights
for layer in nasnet_mobile.layers:
    layer.trainable = False
# our layers - you can add more if you want
x = Flatten()(nasnet_mobile.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=nasnet_mobile.input, outputs=prediction)
# view the structure of the model
#model.summary()

In [13]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/25


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1714751441.255761     131 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1714751441.364211     131 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8371 - loss: 4.8276

W0000 00:00:1714751775.880528     128 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


232/232 ━━━━━━━━━━━━━━━━━━━━ 413s 2s/step - accuracy: 0.8373 - loss: 4.8242 - val_accuracy: 0.8939 - val_loss: 5.4051
Epoch 2/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/25


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


232/232 ━━━━━━━━━━━━━━━━━━━━ 299s 1s/step - accuracy: 0.9352 - loss: 2.2894 - val_accuracy: 0.8204 - val_loss: 13.9862
Epoch 4/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 151us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 298s 1s/step - accuracy: 0.9481 - loss: 1.6802 - val_accuracy: 0.8857 - val_loss: 9.3552
Epoch 6/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 146us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 299s 1s/step - accuracy: 0.9523 - loss: 1.8801 - val_accuracy: 0.8776 - val_loss: 10.9010
Epoch 8/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 146us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 299s 1s/step - accuracy: 0.9743 - loss: 1.0376 - val_accuracy: 0.9020 - val_loss: 11.1577
Epoch 10/25
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 143us/step

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import load_model

2024-05-24 13:57:49.218669: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 13:57:49.218794: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 13:57:49.471060: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
model=load_model('/kaggle/input/model-weight/densnet121.h5')


In [4]:
def classify_image(img):
        try:
            img = keras.preprocessing.image.load_img(img, target_size=(640, 640))
            x=keras.preprocessing.image.img_to_array(img)
            x=x/255
            x=np.expand_dims(x, axis=0)
            prediction= model.predict(x)
            prediction=np.argmax(prediction, axis=1)
            if prediction[0]==0:
                return "This images contains drone"
            else:
                return "This image does not have a drone"
        except:
             return "Facing an Error"

In [5]:
classify_image('/kaggle/input/drone-data/test/102337_jpg.rf.71141a6d3698586e1bd4d4a0feb81553.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step


I0000 00:00:1716559250.952641     112 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


'This images contains drone'